In [2]:
import numpy as np
import json
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from collections import Counter

In [3]:
#load logits
catc = np.load('catboost_results_cluster.npz')#0.78
lgbc = np.load('cluster_lgb_latest_results.npz')#0.85
lgb = np.load('lgb_latest_results.npz')#0.85
xgb = np.load('new_xgb_preds_results.npz')#0.86
log = np.load('logistic_new_results.npz')#0.77


In [21]:
catc_val_pred = np.array(catc['val_preds']).reshape(-1,1)
lgb_val_pred = np.array(lgb['val_preds']).reshape(-1,1)
xgb_val_pred = np.array(xgb['val_preds']).reshape(-1,1)
log_val_pred = np.array(log['val_preds']).reshape(-1,1)

In [29]:
print(catc_val_pred,lgb_val_pred,xgb_val_pred,log_val_pred)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]] [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]] [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]] [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [5]:
catc.files,lgbc.files,lgb.files,xgb.files,log.files

(['val_target',
  'val_preds',
  'feature_importance',
  'test_target',
  'test_preds'],
 ['val_target', 'val_preds', 'test_target', 'test_preds'],
 ['val_target', 'val_preds', 'test_target', 'test_preds'],
 ['val_target',
  'val_preds',
  'test_target',
  'new_test_preds',
  'feature_importance'],
 ['val_target', 'val_preds', 'test_target', 'test_preds'])

In [8]:
catc['feature_importance']

array([ 0.69249275,  9.39362615, 23.50958754,  9.45894704,  7.80315045,
        3.86556479, 12.88759232,  1.21823341,  1.88639708,  7.07852164,
        0.56300667,  0.77477691,  5.40516911,  0.79941537,  0.73149508,
        0.38111232,  0.55084617,  4.9399306 ,  1.37786951,  1.95686375,
        2.17598379,  2.54941752])

In [30]:
score = np.array([0.78,0.85,0.86,0.77])
score_avg = score/np.sum(score)
score_avg

array([0.2392638 , 0.2607362 , 0.26380368, 0.23619632])

In [10]:
print(catc['val_preds'].shape,lgbc['val_preds'].shape,lgb['val_preds'].shape,xgb['val_preds'].shape,log['val_preds'].shape)

(91780,) (91780,) (91780,) (91780,) (91780,)


In [11]:
true_val_y=catc['val_target']
true_test_y=catc['test_target']

In [9]:
def pred(x):
    x_pred = 1 / (1 + np.exp(-x))
    x_pred[x_pred>=0.5]=1
    x_pred[x_pred<0.5]=0
    return x_pred

In [69]:
# weighted vote on [0,1] results val_preds
#all_preds = score_avg[0]*catc_val_pred+score_avg[1]*lgb_val_pred+score_avg[2]*xgb_val_pred+ score_avg[3]*log_val_pred
all_preds = 0.4*lgb['val_preds']+0.6*xgb['val_preds']

In [70]:
print(classification_report(true_val_y,all_preds>0.5))
print(roc_auc_score(true_val_y,all_preds))

              precision    recall  f1-score   support

         0.0       0.77      0.79      0.78     45023
         1.0       0.79      0.77      0.78     46757

    accuracy                           0.78     91780
   macro avg       0.78      0.78      0.78     91780
weighted avg       0.78      0.78      0.78     91780

0.8603590603914353


In [52]:
print(classification_report(lgb['val_target'],lgb['val_preds']>0.5))
print(roc_auc_score(lgb['val_target'], lgb['val_preds']))

              precision    recall  f1-score   support

         0.0       0.76      0.78      0.77     45023
         1.0       0.78      0.77      0.78     46757

    accuracy                           0.77     91780
   macro avg       0.77      0.77      0.77     91780
weighted avg       0.77      0.77      0.77     91780

0.8526378210313116


In [34]:
#majority vote
all_preds_concat = np.concatenate((catc_val_pred,
                                   lgb_val_pred,
                                   xgb_val_pred,
                                   log_val_pred),axis=1)
all_preds_concat.shape

(91780, 4)

In [38]:
all_preds_vote = np.sum(all_preds_concat,axis=1)
all_preds_vote

array([4., 4., 4., ..., 4., 4., 4.])

In [39]:
all_preds_vote[all_preds_vote<=2]=0
all_preds_vote[all_preds_vote>2]=1

In [40]:
print(classification_report(true_val_y,all_preds_vote>0.5))
print(roc_auc_score(true_val_y, all_preds_vote))

              precision    recall  f1-score   support

         0.0       0.75      0.82      0.78     45023
         1.0       0.81      0.74      0.77     46757

    accuracy                           0.78     91780
   macro avg       0.78      0.78      0.78     91780
weighted avg       0.78      0.78      0.78     91780

0.7786748608000571


In [30]:
# #weighted logits prediction
# logits = f1_weight[0]*final_logits+f1_weight[1]*janet_logits+\
#             f1_weight[2]*carson_logits+f1_weight[3]*joyce_logits+ f1_weight[4]*dense_logits #(n_pic,n_class)
# #for [0,1]
# all_preds = pred(logits)
# f1_eval = f1_score(Final['test_true_ans'][:8600], all_preds, average='weighted')
# f1_each = f1_score(Final['test_true_ans'][:8600], all_preds, average=None)

In [31]:
# print(f1_eval,f1_each)

0.9404482344779621 [0.93642384 0.897343   0.93625771 0.944      0.97088693 0.94566254
 0.94470353 0.95290102 0.91307324 0.94414169 0.90879479 0.97286296
 0.94688222]


In [23]:
y_true = [np.argmax(t) for t in  Final['test_true_ans']]
y_pred = [np.argmax(t) for t in all_preds]
y_true = np.array(y_true)
y_pred = np.array(y_pred)
print(y_true.shape,y_pred.shape)
print(y_true[:10],y_pred[:10])
#print(y_true[10:20], Final['test_true_ans'][10:20])
#print(y_pred[10:20], all_preds[10:20])

(8645,) (8645,)
[7 7 7 7 7 7 7 7 7 7] [7 7 7 7 7 7 7 7 7 7]


In [25]:
labels = ['empty', 'opossum', 'bobcat','deer','squirrel', 'rabbit','raccoon', 'coyote', 'fox','cat', 'skunk', 'rodent','dog']
print(classification_report(y_true, y_pred, target_names=labels))

              precision    recall  f1-score   support

       empty       0.80      0.95      0.87       750
     opossum       0.82      0.99      0.90       750
      bobcat       0.96      0.91      0.94       750
        deer       0.94      0.94      0.94       750
    squirrel       0.99      0.96      0.97       750
      rabbit       0.99      0.91      0.95       548
     raccoon       0.95      0.95      0.95       750
      coyote       0.97      0.93      0.95       750
         fox       0.94      0.89      0.91       750
         cat       0.97      0.92      0.94       750
       skunk       0.98      0.86      0.91       328
      rodent       0.99      0.96      0.97       750
         dog       0.98      0.92      0.95       269

   micro avg       0.93      0.93      0.93      8645
   macro avg       0.94      0.93      0.94      8645
weighted avg       0.94      0.93      0.93      8645



In [27]:
print(confusion_matrix(y_true, y_pred))

[[713  13   1  10   0   0   4   0   2   7   0   0   0]
 [  1 743   0   1   3   0   1   0   0   0   1   0   0]
 [ 14   8 685   2   0   3   6   9  17   3   0   2   1]
 [ 23  10   1 707   0   0   4   0   2   2   1   0   0]
 [  8  19   0   1 718   1   2   0   0   0   1   0   0]
 [  7  32   6   2   0 497   0   0   2   0   0   0   2]
 [ 11  14   1   4   0   0 709   0  11   0   0   0   0]
 [  9  29   7   1   0   0   1 696   2   3   0   2   0]
 [ 27  18   2   9   2   0  12   3 669   6   2   0   0]
 [ 32   3   5   8   0   0   0   5   2 692   0   3   0]
 [ 24   4   2   5   1   1   6   0   2   1 281   0   1]
 [ 11   8   2   1   0   0   3   2   2   3   1 717   0]
 [ 10   6   0   1   1   0   2   0   2   0   0   0 247]]
